In [76]:
import yfinance as yf 
from yfinance import EquityQuery
import numpy as np
import pandas as pd

size = 250
rsi_window = 14
roc_window = 5
volatility_window = 30

In [77]:
# Defining a screen for US stocks in specific sectors with certain criteria -- presecreen for options volatility 
screen = EquityQuery("and",[EquityQuery("is-in", ["region", "us"]), EquityQuery("is-in", ["sector",
    "Technology",
    "Financial Services",
    "Consumer Cyclical",
    "Communication Services",
    "Basic Materials",
    "Industrials"
]), EquityQuery("GTE", ["eodprice",0.50])
,   EquityQuery("GTE", ["avgdailyvol3m", 100000]), 
EquityQuery("LTE", ["short_percentage_of_float.value", 100])])

screener = yf.screen(screen,sortField="avgdailyvol3m", sortAsc=False, size = size)

stocks = screener.get('quotes', [])


In [78]:
stocks = pd.DataFrame(stocks)
stocks = stocks.drop(columns=[
    'esgPopulated',
    'cryptoTradeable',
    'hasPrePostMarketData',
    'firstTradeDateMilliseconds',
    'priceHint',
    'regularMarketDayHigh',
    'regularMarketDayRange',
    'regularMarketDayLow',
    'regularMarketPreviousClose',
    'messageBoardId',
    'fullExchangeName',
    'financialCurrency',
    'regularMarketOpen',
    'averageDailyVolume3Month',
    'averageDailyVolume10Day',
    'corporateActions',
    'fiftyTwoWeekLowChange',
    'fiftyTwoWeekLowChangePercent',
    'fiftyTwoWeekRange',
    'fiftyTwoWeekHighChange',
    'fiftyTwoWeekHighChangePercent',
    'fiftyTwoWeekChangePercent',
    'earningsTimestamp',
    'earningsTimestampStart',
    'earningsTimestampEnd',
    'earningsCallTimestampStart',
    'earningsCallTimestampEnd',
    'dividendRate',
    'trailingAnnualDividendYield',
    'marketState',
    'epsCurrentYear',
    'fiftyDayAverage',
    'fiftyDayAverageChange',
    'fiftyDayAverageChangePercent',
    'twoHundredDayAverage',
    'twoHundredDayAverageChange',
    'twoHundredDayAverageChangePercent','dividendYield','gmtOffSetMilliseconds','bid','ask','currency', 'quoteType','language','typeDisp','quoteSourceName','triggerable','customPriceAlertConfidence','sourceInterval','exchangeDataDelayedBy','exchangeTimezoneName','exchangeTimezoneShortName',
 'bidSize','askSize','market','isEarningsDateEstimate','trailingAnnualDividendRate','regularMarketTime','priceEpsCurrentYear','epsForward','forwardPE','tradeable','displayName','longName'
]
)
df = stocks

In [79]:
order = [
    # Identification
    'symbol',
    'shortName',
    'exchange',
    'region',
    
    # Market Price & Change
    'regularMarketPrice',
    'regularMarketChange',
    'regularMarketChangePercent',
    'regularMarketVolume',
    
    # Valuation Metrics
    'marketCap',
    'sharesOutstanding',
    'bookValue',
    'priceToBook',
    'trailingPE',
    'epsTrailingTwelveMonths',
    'averageAnalystRating',
    
    # Price Range
    'fiftyTwoWeekHigh',
    'fiftyTwoWeekLow',
    
    # Dividend Info
    'dividendDate'
]
df = df[order]
df


,symbol,shortName,exchange,region,regularMarketPrice,regularMarketChange,regularMarketChangePercent,regularMarketVolume,marketCap,sharesOutstanding,bookValue,priceToBook,trailingPE,epsTrailingTwelveMonths,averageAnalystRating,fiftyTwoWeekHigh,fiftyTwoWeekLow,dividendDate
0,NVDA,NVIDIA Corporation,NMS,US,135.13,-4.060000,-2.916870,323313507,3394506063872,24387600384,3.438,39.304830,44.893690,3.01,1.4 - Strong Buy,153.13,86.620,1.743552e+09
1,F,Ford Motor Company,NYQ,US,10.38,0.160000,1.565560,132007518,40640294912,3905700096,11.225,0.924722,8.173228,1.27,3.0 - Hold,14.85,8.440,1.748822e+09
2,TSLA,"Tesla, Inc.",NMS,US,346.46,-11.970000,-3.339560,122775188,1154487812096,3220960000,23.184,14.943927,203.799990,1.70,2.6 - Hold,488.54,167.410,NaN
3,LCID,"Lucid Group, Inc.",NMS,US,2.23,-0.180000,-7.468880,254714279,7351128064,3050259968,1.044,2.136015,NaN,-1.10,3.1 - Hold,4.43,1.930,NaN
4,PLTR,Palantir Technologies Inc.,NMS,US,131.78,9.460000,7.733810,182153344,298206265344,2262909952,2.299,57.320570,527.120000,0.25,3.1 - Hold,133.49,21.005,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,SYF,Synchrony Financial,NYQ,US,57.65,-0.599998,-1.030040,5392569,22173286400,380656992,40.361,1.428359,7.984765,7.22,2.0 - Buy,70.93,40.545,1.747267e+09
246,ZETA,Zeta Global Holdings Corp.,NYQ,US,13.14,0.050000,0.381972,4402454,3095048192,211448992,2.866,4.584787,NaN,-0.25,NaN,38.20,10.690,NaN
247,JCI,Johnson Controls International,NYQ,US,101.37,-0.769997,-0.753864,6829361,67212976128,658046976,24.018,4.220585,30.905489,3.28,2.0 - Buy,102.36,64.310,1.744848e+09
248,ETSY,"Etsy, Inc.",NMS,US,55.35,2.360000,4.453660,7094382,5772008448,104282000,-8.659,-6.392193,38.437496,1.44,2.7 - Hold,66.93,40.050,NaN


In [80]:
# Get the price history for the stocks, to do RSI, or MACD, options volatility, and kalman filter acceleration(or see tview workaround)
stocklist = df.symbol.tolist()
stocklist

# Get the price history for the stocks
price_data = yf.download(stocklist, period='14d', interval='1d')
price_data = price_data['Close']

# Calculate the daily returns
change_data = price_data[stocklist]/price_data[stocklist].shift(1)-1

# RSI
change_data
def rsi(price_data, period=rsi_window):
    delta = change_data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))
rsi_data = rsi(price_data)
rsi_data

rsi = round(rsi_data.iloc[-1],1)
df['RSI'] = rsi.values


## Rate of change 
window=5 
recent_avg = change_data.rolling(window=roc_window).mean()
earlier_avg = change_data.shift(window).rolling(window=roc_window).mean()

# Acceleration: difference between recent and earlier averages
accel = (recent_avg - earlier_avg) * 100
accel = round(accel.iloc[-1], 2)
df['ROC'] = accel.values


# Apply filters

filterdf = df[(df['RSI'] < 60) ] # Filter for RSI < 60

filterdf



[*********************100%***********************]  250 of 250 completed

2 Failed downloads:
['INTC']: Timeout('Failed to perform, curl: (28) Connection timed out after 10006 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
['TXN']: Timeout('Failed to perform, curl: (28) Connection timed out after 10004 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_41866/2194723550.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RSI'] = rsi.values
/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_41866/2194723550.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

,symbol,shortName,exchange,region,regularMarketPrice,regularMarketChange,regularMarketChangePercent,regularMarketVolume,marketCap,sharesOutstanding,bookValue,priceToBook,trailingPE,epsTrailingTwelveMonths,averageAnalystRating,fiftyTwoWeekHigh,fiftyTwoWeekLow,dividendDate,RSI,ROC
0,NVDA,NVIDIA Corporation,NMS,US,135.13,-4.060000,-2.916870,323313507,3394506063872,24387600384,3.438,39.304830,44.893690,3.01,1.4 - Strong Buy,153.13,86.620,1.743552e+09,36.6,0.67
1,F,Ford Motor Company,NYQ,US,10.38,0.160000,1.565560,132007518,40640294912,3905700096,11.225,0.924722,8.173228,1.27,3.0 - Hold,14.85,8.440,1.748822e+09,55.3,0.47
2,TSLA,"Tesla, Inc.",NMS,US,346.46,-11.970000,-3.339560,122775188,1154487812096,3220960000,23.184,14.943927,203.799990,1.70,2.6 - Hold,488.54,167.410,NaN,41.6,0.46
3,LCID,"Lucid Group, Inc.",NMS,US,2.23,-0.180000,-7.468880,254714279,7351128064,3050259968,1.044,2.136015,NaN,-1.10,3.1 - Hold,4.43,1.930,NaN,44.9,-3.37
4,PLTR,Palantir Technologies Inc.,NMS,US,131.78,9.460000,7.733810,182153344,298206265344,2262909952,2.299,57.320570,527.120000,0.25,3.1 - Hold,133.49,21.005,NaN,49.5,2.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,AI,"C3.ai, Inc.",NYQ,US,26.59,-1.210000,-4.352510,11259516,3690133760,129239000,6.292,4.226001,NaN,-2.14,3.1 - Hold,45.08,17.030,NaN,48.3,4.09
245,SYF,Synchrony Financial,NYQ,US,57.65,-0.599998,-1.030040,5392569,22173286400,380656992,40.361,1.428359,7.984765,7.22,2.0 - Buy,70.93,40.545,1.747267e+09,45.5,1.09
246,ZETA,Zeta Global Holdings Corp.,NYQ,US,13.14,0.050000,0.381972,4402454,3095048192,211448992,2.866,4.584787,NaN,-0.25,NaN,38.20,10.690,NaN,45.3,0.79
247,JCI,Johnson Controls International,NYQ,US,101.37,-0.769997,-0.753864,6829361,67212976128,658046976,24.018,4.220585,30.905489,3.28,2.0 - Buy,102.36,64.310,1.744848e+09,43.5,0.46


#"""Steps:
#1. get options volatility, prices, data 
#2. get RSI, kalman filter acceleration  (done)
#indicators in screener: filter by RSI, filter by relative options volatility (vs historical volatility, as yfinance does not have historical options volatility data)
#"""
# other:
#take steps to pull all yahoo screen companies 

In [81]:
filterstocks = filterdf.symbol.tolist()
price_datayear = yf.download(filterstocks, period='3mo', interval='1d')
#^^^ fix 
# Historical volatiltiy calculation
# Next add the vol smile relative to HV 
def hv(price_datayear, window=volatility_window):
    log_returns = np.log(price_datayear / price_datayear.shift(1))
    volatility = log_returns.rolling(window=window).std() * np.sqrt(252)  # Annualized volatility
    return volatility

hvol = hv(price_datayear)

[*********************100%***********************]  242 of 242 completed
/opt/anaconda3/lib/python3.12/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


In [82]:
filterstocks = filterdf.symbol.tolist()
#  1. Download price data (make sure columns are tickers)
price_datayear = yf.download(filterstocks, period='3mo', interval='1d')['Close']

# 2. Calculate HV
hvol_latest = price_datayear.apply(hv).iloc[-1]
hvol_latest.name = 'hvol'

# 3. Prepare for merge
filterdf['symbol'] = filterdf['symbol'].astype(str).str.upper()
hvol_latest.index = hvol_latest.index.astype(str).str.upper()

# 4. Merge
filterdf = filterdf.merge(hvol_latest, left_on='symbol', right_index=True, how='left')


[*********************100%***********************]  242 of 242 completed
/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_41866/1090464252.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filterdf['symbol'] = filterdf['symbol'].astype(str).str.upper()


In [83]:
import yfinance as yf
import pandas as pd

def find_closest_value(data_list, target_value):
    """Find the value in a list or Series that is closest to a given target value."""
    if len(data_list) == 0:
        return None
    return min(data_list, key=lambda x: abs(x - target_value))

def filter_special_expiry(expiry_list):
    """
    Returns a list of expiries filtered to only the 2nd lowest, middle, 3rd to last, and last.
    """
    sorted_expiries = sorted(expiry_list)
    length = len(sorted_expiries)
    if length == 0:
        return []
    values_to_keep = set()
    if length > 1:
        values_to_keep.add(sorted_expiries[1])  # 2nd lowest
    values_to_keep.add(sorted_expiries[length // 2])  # middle
    if length >= 3:
        values_to_keep.add(sorted_expiries[-3])  # 3rd to last
    values_to_keep.add(sorted_expiries[-1])  # last
    return list(values_to_keep)

def get_options(symbol):
    try:
        stock = yf.Ticker(symbol)
        options_chain = stock.options
        if not options_chain:
            return pd.DataFrame()
        # Filter expiries first
        filtered_expiries = filter_special_expiry(options_chain)
        options_data = []
        current_price = stock.history(period='1d')['Close'].iloc[-1]
        for expiry in filtered_expiries:
            chain = stock.option_chain(expiry)
            for opt_type, df in [('call', chain.calls), ('put', chain.puts)]:
                if not df.empty:
                    closest_strike = find_closest_value(df['strike'], current_price)
                    filtered = df[df['strike'] == closest_strike].copy()
                    filtered['type'] = opt_type
                    filtered['expiry'] = expiry
                    filtered['symbol'] = symbol
                    options_data.append(filtered)
        if options_data:
            return pd.concat(options_data, ignore_index=True)
        else:
            return pd.DataFrame()
    except Exception as e:
        print(f"Error fetching options for {symbol}: {e}")
        return pd.DataFrame()

# Example usage:
# filterstocks = ['AAPL', 'MSFT', 'NVDA', ...]
options_list = [get_options(symbol) for symbol in filterstocks[1:3]] ### Change this to filterstocks to get all options

options = pd.concat(options_list, ignore_index=True)
options = options[options["openInterest"]>100]
options

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,type,expiry,symbol
0,F271217C00010000,2025-05-30 19:41:11+00:00,10.0,2.11,2.02,2.20,0.100000,4.975120,40.0,1728,0.311774,True,REGULAR,USD,call,2027-12-17,F
1,F271217P00010000,2025-05-30 19:30:55+00:00,10.0,2.04,2.00,2.32,-0.080000,-3.773582,1.0,394,0.392584,False,REGULAR,USD,put,2027-12-17,F
2,F250613C00010500,2025-05-30 19:56:56+00:00,10.5,0.19,0.17,0.19,0.070000,58.333336,2790.0,3842,0.300788,False,REGULAR,USD,call,2025-06-13,F
3,F250613P00010500,2025-05-30 19:50:55+00:00,10.5,0.24,0.26,0.30,-0.160000,-40.000004,230.0,1265,0.289070,True,REGULAR,USD,put,2025-06-13,F
4,F261218C00010000,2025-02-14 20:54:36+00:00,10.0,1.31,1.22,1.38,0.010000,0.769230,16.0,1665,0.233650,True,REGULAR,USD,call,2026-12-18,F
5,F261218P00010000,2025-02-12 17:52:14+00:00,10.0,2.15,1.81,2.17,0.000000,0.000000,85.0,8092,0.471685,False,REGULAR,USD,put,2026-12-18,F
8,TSLA250613C00347500,2025-05-30 19:59:46+00:00,347.5,16.44,16.25,16.45,-8.559999,-34.239998,887.0,181,0.622440,False,REGULAR,USD,call,2025-06-13,TSLA
9,TSLA250613P00347500,2025-05-30 19:58:53+00:00,347.5,17.00,17.40,17.60,4.050000,31.274134,837.0,376,0.626591,True,REGULAR,USD,put,2025-06-13,TSLA
10,TSLA271217C00350000,2025-05-30 19:59:02+00:00,350.0,141.00,140.30,140.95,-10.210007,-6.752203,196.0,1325,0.672634,False,REGULAR,USD,call,2027-12-17,TSLA
11,TSLA271217P00350000,2025-05-30 19:59:58+00:00,350.0,113.50,113.10,113.80,2.500000,2.252252,218.0,1312,0.517980,True,REGULAR,USD,put,2027-12-17,TSLA


In [84]:
## hvol, filterdf, options 
# After filtering for open interest
options = options[options["openInterest"] > 100]

# Calculate average implied volatility for each symbol
result = options.groupby('symbol')['impliedVolatility'].mean().reset_index()

print(result)


  symbol  impliedVolatility
0      F           0.333259
1   TSLA           0.611868
